In [ ]:
!pip install aicrowd-cli

In [ ]:
%load_ext aicrowd.magic
%aicrowd login
!mkdir data
%aicrowd ds dl -c htrec-2022 -o data

In [ ]:
!pip install transformers datasets
#!/usr/bin/env python
# coding=utf-8
# Copyright 2021 The HuggingFace Team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for sequence to sequence.
"""
# You can also adapt this script on your own sequence to sequence task. Pointers for this are left as comments.

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import nltk  # Here to have a nice missing dependency error message early on
import numpy as np
from datasets import load_dataset, load_metric

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    MBart50Tokenizer,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, is_offline_mode, send_example_telemetry
from transformers.utils.versions import require_version


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
# check_min_version("4.21.0.dev0")

# require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/summarization/requirements.txt")

logger = logging.getLogger(__name__)

try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    if is_offline_mode():
        raise LookupError(
            "Offline mode: run this script without TRANSFORMERS_OFFLINE first to download nltk data files"
        )
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)

# A list of all multilingual tokenizer which require lang attribute.
MULTILINGUAL_TOKENIZERS = [MBartTokenizer, MBartTokenizerFast, MBart50Tokenizer, MBart50TokenizerFast]


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": (
                "Will use the token generated when running `transformers-cli login` (necessary to use this script "
                "with private models)."
            )
        },
    )
    resize_position_embeddings: Optional[bool] = field(
        default=None,
        metadata={
            "help": (
                "Whether to automatically resize the position embeddings if `max_source_length` exceeds "
                "the model's position embeddings."
            )
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    lang: str = field(default=None, metadata={"help": "Language id for summarization."})

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    text_column: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the column in the datasets containing the full texts (for summarization)."},
    )
    target_column: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the column in the datasets containing the summaries (for summarization)."},
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a jsonlines or csv file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "An optional input evaluation data file to evaluate the metrics (rouge) on (a jsonlines or csv file)."
            )
        },
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input test data file to evaluate the metrics (rouge) on (a jsonlines or csv file)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_source_length: Optional[int] = field(
        default=1024,
        metadata={
            "help": (
                "The maximum total input sequence length after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    max_target_length: Optional[int] = field(
        default=128,
        metadata={
            "help": (
                "The maximum total sequence length for target text after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded."
            )
        },
    )
    val_max_target_length: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "The maximum total sequence length for validation target text after tokenization. Sequences longer "
                "than this will be truncated, sequences shorter will be padded. Will default to `max_target_length`."
                "This argument is also used to override the ``max_length`` param of ``model.generate``, which is used "
                "during ``evaluate`` and ``predict``."
            )
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": (
                "Whether to pad all samples to model maximum sentence length. "
                "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
                "efficient on GPU but very bad for TPU."
            )
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of training examples to this "
                "value if set."
            )
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
                "value if set."
            )
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of prediction examples to this "
                "value if set."
            )
        },
    )
    num_beams: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "Number of beams to use for evaluation. This argument will be passed to ``model.generate``, "
                "which is used during ``evaluate`` and ``predict``."
            )
        },
    )
    ignore_pad_token_for_loss: bool = field(
        default=True,
        metadata={
            "help": "Whether to ignore the tokens corresponding to padded labels in the loss computation or not."
        },
    )
    source_prefix: Optional[str] = field(
        default="", metadata={"help": "A prefix to add before every source text (useful for T5 models)."}
    )

    forced_bos_token: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "The token to force as the first generated token after the decoder_start_token_id."
                "Useful for multilingual models like mBART where the first generated token"
                "needs to be the target language token (Usually it is the target language token)"
            )
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        if self.val_max_target_length is None:
            self.val_max_target_length = self.max_target_length


summarization_name_mapping = {
    "amazon_reviews_multi": ("review_body", "review_title"),
    "big_patent": ("description", "abstract"),
    "cnn_dailymail": ("article", "highlights"),
    "orange_sum": ("text", "summary"),
    "pn_summary": ("article", "summary"),
    "psc": ("extract_text", "summary_text"),
    "samsum": ("dialogue", "summary"),
    "thaisum": ("body", "summary"),
    "xglue": ("news_body", "news_title"),
    "xsum": ("document", "summary"),
    "wiki_summary": ("article", "highlights"),
}

In [ ]:
!pip install pywer
import pywer
import pandas as pd
import numpy as np
import os
train_df = pd.read_csv("data/train.csv")
print(train_df.shape)
train_df.head()

In [ ]:
test_df = pd.read_csv("data/test.csv")
print(test_df.shape)
test_df.head()

In [ ]:
# ht_raw = " ".join(train_df.HUMAN_TRANSCRIPTION.to_list())
# st_raw = " ".join(train_df.SYSTEM_TRANSCRIPTION.to_list())


# tokens = ht_raw.split()
# WORDS = set(tokens)


# def eddi(input_text, reference_words=WORDS, ed_threshold=25, max_unk_tokens=3):
#   """ Baseline I: Edit distance -based Baseline
#   An edit distance-based baseline: Given a list of valid (reference) words,
#   this baseline (called eddi) detects words not in the reference list and 
#   changes them to the closest one in the reference list.
#   :param input_text: the source text
#   :param reference_words: a list of valid words (e.g., computed from the target data) 
#   :param ed_threshold: the edit distance threshold below from which a word is replaced
#   :param max_unk_tokens: the max number of unknown tokens in the transcribed text 
#   :return: the new text
#   """
#   tokens = input_text.split()
#   # Unknown transcribed tokens; proceed only if few
#   unknowns = [i for i, w in enumerate(tokens) if w not in reference_words]
#   if len(unknowns) > max_unk_tokens:
#     return " ".join(tokens)

#   for ind in unknowns:
#     # Replace each uknown token with the ground truth token w/min edit distance 
#     word = tokens[ind]
#     min_cer, new_word = 100, word
#     for ref in reference_words:
#       candidate_min_cer = pywer.cer([ref], [word])
#       if candidate_min_cer < min_cer:
#         min_cer = candidate_min_cer
#         if min_cer < ed_threshold:
#           new_word = ref
#     tokens[ind] = new_word
#   return " ".join(tokens)




# # Predict for training set
# train_df["B1"] = train_df.SYSTEM_TRANSCRIPTION.apply(eddi)

# # Calculate CER for baseline predictions
# train_df["B1_CER"] = train_df.apply(lambda row: pywer.cer([row.HUMAN_TRANSCRIPTION], [row.B1]), axis=1)
# print(f"B1 CER: {train_df.B1_CER.mean()}")

# # Computing the character error *reduction* rate (CERR)
# train_df["CER"] = train_df.apply(lambda row: pywer.cer([row.HUMAN_TRANSCRIPTION], [row.SYSTEM_TRANSCRIPTION]), axis=1)
# print("Starter CER", train_df.CER.mean())
# print(f"B1 CERR: {(train_df.CER - train_df.B1_CER).mean()}")

In [ ]:
train_df[["SYSTEM_TRANSCRIPTION", "HUMAN_TRANSCRIPTION"]].to_csv("train.csv", index=None)
test_df.assign(HUMAN_TRANSCRIPTION="test")[["SYSTEM_TRANSCRIPTION", "HUMAN_TRANSCRIPTION"]].to_csv("test.csv", index=None)
!head train.csv
!head test.csv

In [ ]:
!pip install accelerate nvidia-ml-py3

In [ ]:
%%file config.json
{
    "model_name_or_path": "google/byt5-large",
    "do_train": true,
    "train_file": "train.csv",
    "test_file": "test.csv",
    "text_column": "SYSTEM_TRANSCRIPTION",
    "target_column": "HUMAN_TRANSCRIPTION",
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "logging_steps": 2,
    "do_predict": true,
    "output_dir": "model",
    "overwrite_output_dir": true,
    "gradient_accumulation_steps": 4,
    "gradient_checkpointing": true,
    "fp16": true
}

    # "fp16": true,
    # "weight_decay": 1e-5,

In [ ]:
%%file config.json
{
    "model_name_or_path": "google/byt5-large",
    "do_train": true,
    "train_file": "train.csv",
    "test_file": "test.csv",
    "text_column": "SYSTEM_TRANSCRIPTION",
    "target_column": "HUMAN_TRANSCRIPTION",
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "logging_steps": 16,
    "do_predict": true,
    "output_dir": "model",
    "overwrite_output_dir": true,
    "gradient_accumulation_steps": 4,
    "gradient_checkpointing": true,
    "optim": "adafactor",
    "learning_rate": 1e-4,
    "seed": 28,
    "warmup_steps": 50,
    "num_train_epochs": 1,
    "lr_scheduler_type": "linear",
    "save_steps": 1000
}

In [ ]:
# See all possible arguments in src/transformers/training_args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, Seq2SeqTrainingArguments))
# if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
    # If we pass only one argument to the script and it's the path to a json file,
    # let's parse it to get our arguments.
    # model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
  # model_args, data_args, training_args = parser.parse_json_file(json_file="config.json")
model_args, data_args, training_args = parser.parse_json_file(json_file="config.json")
# else:
    # model_args, data_args, training_args = parser.parse_args_into_dataclasses()

# Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
# information sent is the one passed as arguments along with your Python/PyTorch versions.
# send_example_telemetry("run_summarization", model_args, data_args)


In [ ]:
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

# if data_args.source_prefix is None and model_args.model_name_or_path in [
#     "t5-small",
#     "t5-base",
#     "t5-large",
#     "t5-3b",
#     "t5-11b",
# ]:
#     logger.warning(
#         "You're running a t5 model but didn't provide a source prefix, which is the expected, e.g. with "
#         "`--source_prefix 'summarize: ' `"
#     )

# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

# Set seed before initializing model.
set_seed(training_args.seed)

# Get the datasets: you can either provide your own CSV/JSON training and evaluation files (see below)
# or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
# (the dataset will be downloaded automatically from the datasets Hub).
#
# For CSV/JSON files this script will use the first column for the full texts and the second column for the
# summaries (unless you specify column names for this with the `text_column` and `target_column` arguments).
#
# In distributed training, the load_dataset function guarantee that only one local process can concurrently
# download the dataset.
# See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
# https://huggingface.co/docs/datasets/loading_datasets.html.

# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)

model.resize_token_embeddings(len(tokenizer))

if model.config.decoder_start_token_id is None and isinstance(tokenizer, (MBartTokenizer, MBartTokenizerFast)):
    if isinstance(tokenizer, MBartTokenizer):
        model.config.decoder_start_token_id = tokenizer.lang_code_to_id[data_args.lang]
    else:
        model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids(data_args.lang)

if model.config.decoder_start_token_id is None:
    raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

if (
    hasattr(model.config, "max_position_embeddings")
    and model.config.max_position_embeddings < data_args.max_source_length
):
    if model_args.resize_position_embeddings is None:
        logger.warning(
            "Increasing the model's number of position embedding vectors from"
            f" {model.config.max_position_embeddings} to {data_args.max_source_length}."
        )
        model.resize_position_embeddings(data_args.max_source_length)
    elif model_args.resize_position_embeddings:
        model.resize_position_embeddings(data_args.max_source_length)
    else:
        raise ValueError(
            f"`--max_source_length` is set to {data_args.max_source_length}, but the model only has"
            f" {model.config.max_position_embeddings} position encodings. Consider either reducing"
            f" `--max_source_length` to {model.config.max_position_embeddings} or to automatically resize the"
            " model's position encodings by passing `--resize_position_embeddings`."
        )

prefix = data_args.source_prefix if data_args.source_prefix is not None else ""


In [ ]:
if data_args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset(
        data_args.dataset_name,
        data_args.dataset_config_name,
        cache_dir=model_args.cache_dir,
        use_auth_token=True if model_args.use_auth_token else None,
    )
else:
    data_files = {}
    if data_args.train_file is not None:
        data_files["train"] = data_args.train_file
        extension = data_args.train_file.split(".")[-1]
    if data_args.validation_file is not None:
        data_files["validation"] = data_args.validation_file
        extension = data_args.validation_file.split(".")[-1]
    if data_args.test_file is not None:
        data_files["test"] = data_args.test_file
        extension = data_args.test_file.split(".")[-1]
    raw_datasets = load_dataset(
        extension,
        data_files=data_files,
        cache_dir=model_args.cache_dir,
        use_auth_token=True if model_args.use_auth_token else None,
    )

# Preprocessing the datasets.
# We need to tokenize inputs and targets.
if training_args.do_train:
    column_names = raw_datasets["train"].column_names
elif training_args.do_eval:
    column_names = raw_datasets["validation"].column_names
elif training_args.do_predict:
    column_names = raw_datasets["test"].column_names
else:
    logger.info("There is nothing to do. Please pass `do_train`, `do_eval` and/or `do_predict`.")
    # return

if isinstance(tokenizer, tuple(MULTILINGUAL_TOKENIZERS)):
    assert (
        data_args.lang is not None
    ), f"{tokenizer.__class__.__name__} is a multilingual tokenizer which requires --lang argument"

    tokenizer.src_lang = data_args.lang
    tokenizer.tgt_lang = data_args.lang

    # For multilingual translation models like mBART-50 and M2M100 we need to force the target language token
    # as the first generated token. We ask the user to explicitly provide this as --forced_bos_token argument.
    forced_bos_token_id = (
        tokenizer.lang_code_to_id[data_args.forced_bos_token] if data_args.forced_bos_token is not None else None
    )
    model.config.forced_bos_token_id = forced_bos_token_id

# Get the column names for input/target.
dataset_columns = summarization_name_mapping.get(data_args.dataset_name, None)
if data_args.text_column is None:
    text_column = dataset_columns[0] if dataset_columns is not None else column_names[0]
else:
    text_column = data_args.text_column
    if text_column not in column_names:
        raise ValueError(
            f"--text_column' value '{data_args.text_column}' needs to be one of: {', '.join(column_names)}"
        )
if data_args.target_column is None:
    target_column = dataset_columns[1] if dataset_columns is not None else column_names[1]
else:
    target_column = data_args.target_column
    if target_column not in column_names:
        raise ValueError(
            f"--target_column' value '{data_args.target_column}' needs to be one of: {', '.join(column_names)}"
        )

# Temporarily set max_target_length for training.
max_target_length = data_args.max_target_length
padding = "max_length" if data_args.pad_to_max_length else False

if training_args.label_smoothing_factor > 0 and not hasattr(model, "prepare_decoder_input_ids_from_labels"):
    logger.warning(
        "label_smoothing is enabled but the `prepare_decoder_input_ids_from_labels` method is not defined for"
        f"`{model.__class__.__name__}`. This will lead to loss being calculated twice and will take up more memory"
    )

def preprocess_function(examples):
    # remove pairs where at least one record is None

    inputs, targets = [], []
    for i in range(len(examples[text_column])):
        if examples[text_column][i] is not None and examples[target_column][i] is not None:
            inputs.append(examples[text_column][i])
            targets.append(examples[target_column][i])

    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length" and data_args.ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

if training_args.do_train:
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = raw_datasets["train"]
    if data_args.max_train_samples is not None:
        max_train_samples = min(len(train_dataset), data_args.max_train_samples)
        train_dataset = train_dataset.select(range(max_train_samples))
    with training_args.main_process_first(desc="train dataset map pre-processing"):
        train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on train dataset",
        )

if training_args.do_eval:
    max_target_length = data_args.val_max_target_length
    if "validation" not in raw_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_dataset = raw_datasets["validation"]
    if data_args.max_eval_samples is not None:
        max_eval_samples = min(len(eval_dataset), data_args.max_eval_samples)
        eval_dataset = eval_dataset.select(range(max_eval_samples))
    with training_args.main_process_first(desc="validation dataset map pre-processing"):
        eval_dataset = eval_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on validation dataset",
        )

if training_args.do_predict:
    max_target_length = data_args.val_max_target_length
    if "test" not in raw_datasets:
        raise ValueError("--do_predict requires a test dataset")
    predict_dataset = raw_datasets["test"]
    if data_args.max_predict_samples is not None:
        max_predict_samples = min(len(predict_dataset), data_args.max_predict_samples)
        predict_dataset = predict_dataset.select(range(max_predict_samples))
    with training_args.main_process_first(desc="prediction dataset map pre-processing"):
        predict_dataset = predict_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on prediction dataset",
        )

# Data collator
label_pad_token_id = -100 if data_args.ignore_pad_token_for_loss else tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if training_args.fp16 else None,
)

# Metric
# metric = load_metric("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if data_args.ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


In [ ]:
# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics if training_args.predict_with_generate else None,
)

# Training
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    try:
        train_result = trainer.train(resume_from_checkpoint=checkpoint)
    except KeyboardInterrupt:
        print("Early stopping...")
    trainer.save_model()  # Saves the tokenizer too for easy upload

    # metrics = train_result.metrics
    # max_train_samples = (
        # data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    # )
    # metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    # trainer.log_metrics("train", metrics)
    # trainer.save_metrics("train", metrics)
    trainer.save_state()

# Evaluation
results = {}
max_length = (
    training_args.generation_max_length
    if training_args.generation_max_length is not None
    else data_args.val_max_target_length
)
num_beams = data_args.num_beams if data_args.num_beams is not None else training_args.generation_num_beams
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    metrics = trainer.evaluate(max_length=max_length, num_beams=num_beams, metric_key_prefix="eval")
    max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
    metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

In [ ]:
num_beams = 1

In [ ]:
# help(trainer.predict)

In [ ]:
# training_args.predict_with_generate = True

In [ ]:
# tokenizer.batch_decode(predict_dataset.train_test_split(0.01, shuffle=False)["test"]["input_ids"], skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
# for i in predict_dataset:
#     pass

In [ ]:
# print(getsource(trainer.evaluation_loop))

In [ ]:
# print(getsource(trainer.callback_handler.call_event))
# print(getsource(trainer.callback_handler.callbacks[-1].on_prediction_step))
# trainer.callback_handler.callbacks[-1]

In [ ]:
# trainer.callback_handler.callbacks[-1].prediction_bar = None

In [ ]:
# tokenizer.batch_decode(trainer.predict(predict_dataset.train_test_split(0.01, shuffle=False)["test"], max_length=128).predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
!head test.csv

In [ ]:
# if training_args.do_predict:
#     logger.info("*** Predict ***")

#     trainer.callback_handler.callbacks[-1].prediction_bar = None
#     predict_results = trainer.predict(
#         predict_dataset, metric_key_prefix="predict", max_length=max_length, num_beams=None
#     )
#     # metrics = predict_results.metrics
#     # max_predict_samples = (
#         # data_args.max_predict_samples if data_args.max_predict_samples is not None else len(predict_dataset)
#     # )
#     # metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

#     # trainer.log_metrics("predict", metrics)
#     # trainer.save_metrics("predict", metrics)

In [ ]:
from tqdm.auto import tqdm
import torch


gens = []
model.eval()
bar = tqdm(predict_dataset)
dec = lambda x: tokenizer.batch_decode([x], skip_special_tokens=True,
                                       clean_up_tokenization_spaces=True)[0].strip()
for x in bar:
    gen = model.generate(input_ids=torch.tensor(x["input_ids"]).to(model.device).view(1, -1),
                         do_sample=False, max_length=128, num_beams=1)
    bar.set_description(f"{dec(x['input_ids'])} -> {dec(gen[0])}")
    gens.append(dec(gen[0]))
print(gens)

In [ ]:
# [x.shape for x in predict_results.predictions]

In [ ]:
# import torch


# if training_args.do_predict and trainer.is_world_process_zero():
#     predictions = tokenizer.batch_decode(
#         predict_results.predictions[1].argmax(axis=-1), skip_special_tokens=True, clean_up_tokenization_spaces=True
#     )
#     predictions = [pred.strip() for pred in predictions]
#     output_prediction_file = os.path.join(training_args.output_dir, "generated_predictions.txt")
#     with open(output_prediction_file, "w") as writer:
#         writer.write("\n".join(predictions))

In [ ]:
# !head model/generated_predictions.txt
# !head data/test.csv

In [ ]:
from google.colab import files
from datetime import datetime


filename = f"predict_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
pd.DataFrame(list(zip(
    test_df["IMAGE_PATH"].tolist(),
    gens
    # [line[:-1] for line in open("model/generated_predictions.txt")]
    )),
             columns=["ImageID", "Transcriptions"]).to_csv(filename, index=None)
!head {filename}
files.download(filename)